# Lleva a cabo la Validación del Modelo Entrenado con TF2 comparando contra los objetos indicados en los XMLs correspondientes
Se basa en propuesta de https://towardsdatascience.com/evaluating-performance-of-an-object-detection-model-137a349c517b

0) Preparar ambiente e instalar paquetes:

In [ ]:
#@title Clonar el repositorio de modelos de TF si no está ya disponible
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
#@title Instalar el Object Detection API
# Nota: si dice que faltan librerías, ignorar (funciona bien igual) 
#       sino volverlo a ejecutar esta celda para que reinistale y entonces dice todo "successfully"
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .


1) Cargar librerías:

In [ ]:
#@title Cargar Librerías
import os
import os.path
import sys
import numpy as np
import pandas as pd
from random import sample

from IPython.display import Image, display
from PIL import Image as ImPIL

import tensorflow as tf
from PIL import ImageColor
from PIL import ImageDraw

import copy
import xml.etree.cElementTree as ET

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import time

print ("Librerías cargadas.")

2) Montar el Drive:

In [ ]:
#@title Montar Google Drive
# Nota: la primera vez se debe confirmar el uso logueandose en "Google Drive File Stream" y obteniendo código de autentificación.
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
#@title Definir configuración de directorios local en Google Drive
drive_path = '/content/gdrive/My Drive/GEMIS/objDetectionCursogramas' #@param {type:"string"}
drive_subdir_datos = '/Cursogramas' #@param {type:"string"}
drive_subdir_modelo = '/TF_model' #@param {type:"string"}

data_dir_path = drive_path + drive_subdir_datos
model_drive_path = drive_path + drive_subdir_modelo

print("Configuración de archivos definida")

3) Cargar el modelo entrenado:

In [ ]:
#@title Cargar el modelo de object detection entrenado y define funciones auxiliares
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# carga el modelo exportado 
ModelObjDetEntrenado = model_drive_path + '/saved_model'
detection_model = tf.saved_model.load(str(ModelObjDetEntrenado))
print("\nModelo objDetector cargado: [", ModelObjDetEntrenado, "]: ", detection_model)

# archivo con lista de clases para reconocer 
labelMapFile = model_drive_path + '/label_map.pbtxt'

label_map = label_map_util.load_labelmap(labelMapFile)
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=label_map_util.get_max_label_map_index(label_map),
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(label_map, use_display_name=True)
print("\nDefinición de Clases cargada: [", labelMapFile, "]: ", len(category_index))

# Size, in inches, of the output images.
##IMAGE_SIZE = (12, 8)
##print("\nIMAGE SIZE: ",  IMAGE_SIZE)

## funciones auxiliares

# función auxiliar para conversión de la imagen ( NO SE USA )
#def load_image_into_numpy_array(image):
#    (im_width, im_height) = image.size
#    return np.array(image.getdata()).reshape(
#        (im_height, im_width, 3)).astype(np.uint8)

# función auxiliar para procesar la imagen con el modelo
def run_inference_for_single_image(model, image_np):   
    # fuerza conversión a array por las dudas
    image_np = np.asarray(image_np) 
    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis,...]

    # Run inference
    model_fn = model.signatures['serving_default']
    output_dict = model_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(output_dict.pop('num_detections'))
    output_dict = {key:value[0, :num_detections].numpy() 
                  for key,value in output_dict.items()}
    output_dict['num_detections'] = num_detections

    # detection_classes should be ints.
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
    
    # Handle models with masks:
    if 'detection_masks' in output_dict:
      # Reframe the the bbox mask to the image size.
      detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                output_dict['detection_masks'], output_dict['detection_boxes'],
                image.shape[0], image.shape[1])      
      detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                        tf.uint8)
      output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
      
    return output_dict

# función auxiliar para mostrar resultados de procesar la imagen con el modelo
def plot_detections(image_np,
                    boxes,
                    classes,
                    scores,
                    category_index,
                    line_thickness = 8,
                    min_score = 0.8):

    # genera una copia de la imagen
    image_np_with_annotations = image_np.copy()

    # en la copia marca los objetos detectados
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np_with_annotations,
        boxes,
        classes,
        scores,
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=100,
        line_thickness=line_thickness,
        min_score_thresh=min_score,
        agnostic_mode=False)
        
    # muestra la copia de la imagen con los objetos detectados
    display(ImPIL.fromarray(image_np_with_annotations))  
    #print("-- objetos detectados: ", len(classes), "\n")  # siempre son 300

print("\nFunciones Auxiliares definidas.")

4) Llevar a cabo la validación usando las imágenes y XML:

In [ ]:
#@title Definir funciones auxiliares 

# función para cálculo de Intersection over Union (IoU) 
def calc_IoU( gt_bbox, pred_bbox):
    '''
    This function takes the predicted bounding box and ground truth bounding box and 
    return the IoU ratio
    '''
    x_topleft_gt, y_topleft_gt, x_bottomright_gt, y_bottomright_gt = gt_bbox
    x_topleft_p, y_topleft_p, x_bottomright_p, y_bottomright_p = pred_bbox
    
    if (x_topleft_gt > x_bottomright_gt) or (y_topleft_gt> y_bottomright_gt):
        raise AssertionError("Ground Truth Bounding Box is not correct")
    if (x_topleft_p > x_bottomright_p) or (y_topleft_p> y_bottomright_p):
        raise AssertionError("Predicted Bounding Box is not correct",x_topleft_p, x_bottomright_p,y_topleft_p,y_bottomright_gt)
        
         
    #if the GT bbox and predcited BBox do not overlap then iou=0
    if(x_bottomright_gt< x_topleft_p):
        # If bottom right of x-coordinate  GT  bbox is less than or above the top left of x coordinate of  the predicted BBox
        
        return 0.0
    if(y_bottomright_gt< y_topleft_p):  # If bottom right of y-coordinate  GT  bbox is less than or above the top left of y coordinate of  the predicted BBox
        
        return 0.0
    if(x_topleft_gt> x_bottomright_p): # If bottom right of x-coordinate  GT  bbox is greater than or below the bottom right  of x coordinate of  the predcited BBox
        
        return 0.0
    if(y_topleft_gt> y_bottomright_p): # If bottom right of y-coordinate  GT  bbox is greater than or below the bottom right  of y coordinate of  the predcited BBox
        
        return 0.0
    
    
    GT_bbox_area = (x_bottomright_gt -  x_topleft_gt + 1) * (  y_bottomright_gt -y_topleft_gt + 1)
    Pred_bbox_area =(x_bottomright_p - x_topleft_p + 1 ) * ( y_bottomright_p -y_topleft_p + 1)
    
    x_top_left =np.max([x_topleft_gt, x_topleft_p])
    y_top_left = np.max([y_topleft_gt, y_topleft_p])
    x_bottom_right = np.min([x_bottomright_gt, x_bottomright_p])
    y_bottom_right = np.min([y_bottomright_gt, y_bottomright_p])
    
    intersection_area = (x_bottom_right- x_top_left + 1) * (y_bottom_right-y_top_left  + 1)
    
    union_area = (GT_bbox_area + Pred_bbox_area - intersection_area)
   
    return intersection_area/union_area


# función para mostrar boxes en una imagen dada
def draw_box(draw, rangeObj, lineWidth, lineColor):  

    draw.line([(rangeObj[0], rangeObj[1]), 
                        (rangeObj[0], rangeObj[3]), 
                        (rangeObj[2], rangeObj[3]), 
                        (rangeObj[2], rangeObj[1]), 
                        (rangeObj[0], rangeObj[1])], 
                      width=lineWidth, fill=lineColor)

# función para mostrar boxes de una lista objetos en una imagen dada
def draw_boxes_listObj(draw, listObj, lineWidth, lineColor):
  
    for obj in listObj:
        draw_box(draw, obj[2], lineWidth, lineColor)

# función para mostrar métricas de los resultados
def mostrarMetricas(metricas, titulo):

  print("\n= " + titulo + ":")        
  print("                  Modelo ")
  print(" XML   :       +          -   ")
  print("  +    :     %3d        %3d  " % (metricas[posVP], metricas[posFN]) )
  print("  -    :     %3d        %3d  " % (metricas[posFP], metricas[posVN]) )

  # Cálculo de la Exactitud
  total = (metricas[posVP] + metricas[posFP] + metricas[posVN] + metricas[posFN])
  if total>0:
      print("= Exactitud: ", round(100*(metricas[posVP] + metricas[posVN])/total, 3))

  # Cálculo de la Precisión
  total = (metricas[posVP] + metricas[posFP])
  if total>0:
      print("= Precisión: ",  round(100*metricas[posVP]/total,3))

  # Cálculo de la Recuperación
  total = (metricas[posVP] + metricas[posFN])
  if total>0:
      print("= Recuperación: ", round(100*metricas[posVP]/total,3))
  
  print("\n")
  return

print("Funciones auxiliares definidas")   

In [ ]:
#@title Definir imágenes a utilizar
# define la carpeta donde están las imágenes para procesar
imagenes_utilizar = '/validation' #@param [ '/validation', '/Generados' ]
if imagenes_utilizar == '/validation':
  dirTestImg = data_dir_path + '/validation/images' 
  dirTestXML = data_dir_path + '/validation/annotations' 
elif imagenes_utilizar == '/Generados':
  dirTestImg = data_dir_path + '/Generados'
  dirTestXML = data_dir_path + '/Generados'

# levanta los XML de validación para dirTestXML
process_FileNames = [ fn for fn in os.listdir( dirTestXML ) if fn.endswith('.xml') ]
print("> Imágenes/XML a probar: ", len(process_FileNames))

In [ ]:
#@markdown ### Tomar una muestra de las imágenes (si es necesario o se quiere) { run: "auto" }
porcMuestraImagenesProcesar = 100  #@param {type:"slider", min:0, max:100, step:5}
if porcMuestraImagenesProcesar>0 and porcMuestraImagenesProcesar<100:
  cantProcesar = int(len(process_FileNames)*porcMuestraImagenesProcesar/100)
  if cantProcesar==0:
    cantProcesar = 1
  process_FileNames = sample(process_FileNames, cantProcesar)    
print("> Imágenes/XML a probar: ", len(process_FileNames))

In [ ]:
#@title Definir parámetros a utilizar { run: "auto" }

# define minima probabilidad a usar
minimaProbabilidadObjectosDetectados = 90 #@param {type:"slider", min:1, max:100, step:1.0}
minProbObjDet = minimaProbabilidadObjectosDetectados / 100.

# define si muestra detalle o no
muestraDetalleDebug = False  #@param {type:"boolean"}
muestraDetalleObjDetectadosEnImagen = False  #@param {type:"boolean"}
muestraDetalleComparacionEnImagen = "Solo con Error" #@param ["Ninguna", "Solo con Error", "Todas"]
muestraDetalleMetricasPorImagen = "Solo con Error" #@param ["Ninguna", "Solo con Error", "Todas"]
muestraDetalleMetricasPorClaseObjeto = True  #@param {type:"boolean"}

# define parámetro Intersection over Union (IoU) 
## si calc_IoU(r1, r2) ≥ coefIoU, se considera que se detectó el objecto correctamente, es Verdadero Positivo (VP)
## si calc_IoU(r1, r2) < coefIoU, se considera que se detectó el objecto con error, es Falso Positivo (FP)
## -> valor recomendado por defecto: 0,5 
## pero se usa menos para mejorar los resultados
coefIoU = 0.4 #@param {type:"slider", min:0.1, max:1, step:0.1}

print("Parámetros definidos")

In [ ]:
#@title Realizar el Procesamiento de las Imágenes comparando contra XMLs

# inicializa vector auxiliar para metricas y posiciones a usar
listaXMLConProblemas = []
cantXMLProcesados = 0
metricasGral = [ 0, 0, 0, 0]
posVP = 0
posVN = 1
posFP = 2
posFN = 3

# inicializa diccionario auxiliar para metricas por tipo de caso generado
metricasGral_porTipoCaso = {}

# inicializa vectores auxiliares para evaluación de objetos detectados
classObjModelo = []
classObjReal = []

# auxiliar para calcular tiempos del modelo
auxSumaTiempo = 0
auxCantProc = 0

# muestra parámetros
print("> Parámetros: ")
print("  minimaProbabilidadObjectosDetectados: ", minimaProbabilidadObjectosDetectados)
print("  coefIoU: ", coefIoU)
print("\n")
print("  muestraDetalleDebug: ", muestraDetalleDebug)
print("  muestraDetalleObjDetectadosEnImagen: ", muestraDetalleObjDetectadosEnImagen)
print("  muestraDetalleComparacionEnImagen: ", muestraDetalleComparacionEnImagen)
print("  muestraDetalleMetricasPorImagen: ", muestraDetalleMetricasPorImagen)
print("  muestraDetalleMetricasPorClaseObjeto: ", muestraDetalleMetricasPorClaseObjeto)
print("\n\n\n")

# Procesa los XMLs de las imágenes 
for xml_file in process_FileNames:

    # inicializa vectores auxiliares
    listObjsXML = []
    listObjsDetModelo = []
    
    print("\n------------------------------------------------------------------------------------------------------------")
    cantXMLProcesados = cantXMLProcesados + 1
    print("<", cantXMLProcesados, "> ", xml_file, ": ")

    # determina el tipo de archivo
    tipoCaso = ""
    if xml_file.startswith("da_"):
      tipoCaso = "DA"
    else:
      tipoCaso = "OR"
    if xml_file.find("r-")>=0 or xml_file.find("r.")>=0:
      tipoCaso = tipoCaso + '_R'
    elif xml_file.find("s-")>=0 or xml_file.find("s.")>=0:
      tipoCaso = tipoCaso + '_S'
    else:
      tipoCaso = tipoCaso + '_N'

    # carga la info del XML original
    et = ET.parse(dirTestXML + '/' + xml_file)
    element = et.getroot()
    element_objs = element.findall('object') 
    element_filename = element.find('filename').text

    imagenProcesar = os.path.join(dirTestImg, element_filename)

    # controla que exista la imagen
    if not os.path.isfile(imagenProcesar):
      print("\t -- No se encuentra la imagen ", imagenProcesar, "!\n")
      # deja de procesar el XML y pasa al siguiente
      continue

    # carga los elementos en el archivo XML original para generar el nuevo
    for element_obj in element_objs:

        # obtiene la información actual de la imagen
        class_name = element_obj.find('name').text 

        # obtiene info del box actual
        obj_bbox = element_obj.find('bndbox')
        nuevoRangoIm = [ float(obj_bbox.find('xmin').text),
                        float(obj_bbox.find('ymin').text),
                        float(obj_bbox.find('xmax').text), 
                        float(obj_bbox.find('ymax').text) ]

        # si tiene invertidas las posiciones las corrige
        if nuevoRangoIm[0] > nuevoRangoIm[2]:
          auxnuevoRangoIm = nuevoRangoIm[2]
          nuevoRangoIm[2] = nuevoRangoIm[0]
          nuevoRangoIm[0] = auxnuevoRangoIm
        if nuevoRangoIm[1] > nuevoRangoIm[3]:
          auxnuevoRangoIm = nuevoRangoIm[3]
          nuevoRangoIm[3] = nuevoRangoIm[1]
          nuevoRangoIm[1] = auxnuevoRangoIm

        # calcula un valor para poder ordenar las figuras de arriba a abajo y izquierda a derecha
        centroideIm = nuevoRangoIm[1]*100000+nuevoRangoIm[0]

        # agrega a lista de objetos cargados del XML
        # controlando que no estuviera ya (para evitar duplicados)
        elObjXML = (centroideIm, class_name, nuevoRangoIm)        
        if elObjXML not in listObjsXML:
          listObjsXML.append( elObjXML )

      # carga la imagen a procesar
    imageCargada = ImPIL.open(imagenProcesar) 

    # Convierte la imagen a escala de grises y luego a RGB 
    # (para sacarle los colores que tuviera previamente y dejarlo con 3 canales de profundidad)
    imageCargada = imageCargada.convert('L')
    imageCargada = imageCargada.convert('RGB')

    # obtiene el tamaño de la imagen
    imCargada_ancho, imCargada_alto = imageCargada.size

    # convierte la imagen a un array 
    image_np = np.array(imageCargada)

    # Procesa el array de la imagen con el modelo cargado
    tiempoInicioModelo = time.time()
    output_dict = run_inference_for_single_image(detection_model, image_np)
    tiempoDemoraModelo = (time.time() - tiempoInicioModelo)
    if muestraDetalleDebug:
        print("\n# Ejecutar el modelo demora ", tiempoDemoraModelo, " segundos. \n")
    auxSumaTiempo = auxSumaTiempo  + tiempoDemoraModelo
    auxCantProc = auxCantProc  + 1

    # procesa los objetos detectados
    for detClass, detBox, detScore in zip(  output_dict['detection_classes'], output_dict['detection_boxes'], output_dict['detection_scores'] ):

        class_name = category_index[detClass]['name']

        # como las coordenadas están normalizadas las debe convertir 
        # teniendo en cuenta el tamaño de la imagen
        # además notar que vienen datas en otro orden
        # - detBox = (ini alto, ini ancho, fin alto, fin ancho)
        # - nuevoRangoIn = (ini ancho x1, ini alto y1, fin ancho x2, fin alto y2)    
        nuevoRangoIm = [detBox[1] * imCargada_ancho, 
                        detBox[0] * imCargada_alto,
                        detBox[3] * imCargada_ancho,
                        detBox[2] * imCargada_alto]

        # si el objeto detectado tiene un puntaje superior o igual al mínimo
        if detScore >= minProbObjDet:

              # calcula un valor para poder ordenar las figuras de arriba a abajo y izquierda a derecha
              centroideIm = nuevoRangoIm[1]*100000+nuevoRangoIm[0]

              # agrega a lista de objetos detectados por el modelo
              listObjsDetModelo.append( (centroideIm, class_name, nuevoRangoIm) )          
        else:
              if muestraDetalleDebug and detScore >= 0.4:
                print("-- Objeto descartado por bajo score: ", class_name, "(", detScore*100, "%) en ", nuevoRangoIm)

    # ordena por el centroide las dos listas
    listObjsXML = sorted(listObjsXML, key=lambda objDet: objDet[0])  
    listObjsDetModelo = sorted(listObjsDetModelo, key=lambda objDet: objDet[0])  

    if muestraDetalleDebug:
        print("\n- Objetos del XML:")
        print(len(listObjsXML), " : ",listObjsXML)

        print("\n- Objetos detectados por el Modelo:")
        print(len(listObjsDetModelo), " : ", listObjsDetModelo)

    if muestraDetalleObjDetectadosEnImagen: 

        print("\n- Muestra los objetos del XML y Detectados en la Imagen:")
        # imagen auxiliar para mostrar recuadros de XML y modelo
        image_pil = copy.deepcopy(imageCargada.convert("RGB"))
        draw = ImageDraw.Draw(image_pil)
        im_width, im_height = image_pil.size    

        # genera los recuadros correspondientes del XML (en color verde)
        draw_boxes_listObj(draw, listObjsXML, 8, (0,255,0))
            
        # genera los recuadros correspondientes al Modelo (en color azul)
        draw_boxes_listObj(draw, listObjsDetModelo, 4, (0,0,255))

        imMostrar = ImPIL.fromarray(np.array(image_pil), 'RGB')
        display( imMostrar )
        print(" Nota colores:")
        print("    Objetos definidos en el XML, cuadros en VERDE.")
        print("    Objetos detectados por el Modelo, cuadros en AZUL.")
        print("\n")

    # Realiza la compración de los objetos definidos en el XML contra los detectados por el modelo
    if muestraDetalleDebug:
        print("\n+ Realiza la Comparación: ")

    resCompara = []
    auxlistObjsDetModelo = copy.copy(listObjsDetModelo)

    # Busca el objeto del XML en la lista de objetos detectados por el Modelo
    # (para eso considera la ubicación y tipo de clase)
    for objXML in listObjsXML:
        i = -1
        noEnc = True
        while noEnc and i < (len(auxlistObjsDetModelo)-1):         
          i = i + 1
          
          # calcula la Intersection over Union (IoU) de los boxes
          objIoU = calc_IoU( objXML[2], auxlistObjsDetModelo[i][2] )

          # si el IoU es casi perfecto, considera que es el objeto
          #if objIoU > 0.90: 
          #  noEnc = False
          #else:
           #  analiza si tiene algo de superposición y es de la misma clase
          noEnc = not((objIoU > 0.1) and (objXML[1] == auxlistObjsDetModelo[i][1]))

        if noEnc:

           # Si no se encuentra objeto con misma ubicación y clase del XML
          if muestraDetalleDebug:
              print(objXML[1], " no detectado por el Modelo con misma ubicación y clase ")

          # registra que ese objeto no se encontró
          resCompara.append(  (objXML[1], -1, objXML[2], "*") )

        else:

          # Si encuentra objecto en misma ubicación y  clase del XML
          if muestraDetalleDebug:
              print(objXML[1], ": ", objIoU)

          # regista que el objeto se encontró con su IoU
          resCompara.append(  (objXML[1], objIoU, auxlistObjsDetModelo[i][2], auxlistObjsDetModelo[i][1]) )

          # saca el objeto de la lista auxiliar para que no se vuelva a usar
          auxlistObjsDetModelo.pop(i)

    # Revisa los objetos detectados que no se utilizaron en la comparación anterior
    # para incluir en la comparación  
    #  objetos en la misma ubicación pero  con distinta clase
    #   u objetos detectados que no figuran en el XML
    if len(auxlistObjsDetModelo) > 0:
      if muestraDetalleDebug:
        print("\n-Se intenta asociar con ", len(auxlistObjsDetModelo), " objetos detectados del Modelo no utilizados")

      for objDet in auxlistObjsDetModelo:

        # lo compara con los que no se puedieron detectar
        i = 0
        cont = True
        while cont and i < len(resCompara):

            # sólo procesa es un objeto del XML no encontrado en Modelo 
            if (resCompara[i][1] < 0):
                # calcula la Intersection over Union (IoU) de los boxes
                objIoU = calc_IoU( resCompara[i][2], objDet[2] )

                if objIoU >= coefIoU:
                      # si están superpuestos se considera que se detectó pero le asignó mal la clase                    
                      resCompara[i] = (resCompara[i][0], objIoU, objDet[2], objDet[1] )                   
                      cont = False

                      if muestraDetalleDebug:
                          print("--- Se rectifica objeto no detectado: ", resCompara[i])
            i = i + 1

        if cont:
          # si no se utiliza, se agrega como objeto detectado de más
          resCompara.append(  ("*", 999, objDet[2], objDet[1]) )

          if muestraDetalleDebug:
              print("--- Se agrega objeto detectado por Modelo de más: ", objDet[1])

    if muestraDetalleDebug:

        print("\n+ Resultados de la Comparación:")
        print( len(resCompara), " : ", resCompara )


    # Realiza el cálculo de las Métricas de la Imagen considerando los resultados de la comparación
    ## ---------------------------------------------------------------------------------------------
    ## Nota los Verdadero Negativo (VN) se deberían calcular considerando el resto de la imagen que no tiene 
    ## objetos, por lo que no es útil para Modelos de Object Detection y no se utiliza.
    ## ---------------------------------------------------------------------------------------------
    metricasImag = [ 0, 0, 0, 0]

    hayErrorDetectado = False
    generaImagenComparacion = (muestraDetalleComparacionEnImagen != "Ninguna")
    if generaImagenComparacion:
      # imagen auxiliar para mostrar resultados comparación
      comp_image_pil = copy.deepcopy(imageCargada.convert("RGB"))
      comp_draw = ImageDraw.Draw(comp_image_pil)
      im_width, im_height = comp_image_pil.size     

    for resObj in resCompara:

        if muestraDetalleMetricasPorClaseObjeto:
          # agrega en vectores auxiliares para hacer la evaluación por clase
          classObjReal.append( resObj[0] )
          classObjModelo.append( resObj[3] )

        if (resObj[0] == resObj[3]) and (resObj[1] >= coefIoU):

            # Objeto de misma Clase entre XML y Modelo con IoU ≥ coefIoU -> Verdadero Positivo (VP)
            posM = posVP
            if generaImagenComparacion:            
              draw_box(comp_draw, resObj[2], 4, (0,255,0)) # cuadros Verdes

        elif resObj[1] < 0:

            # Objecto del XML no encontrado en Modelo -> Falso Negativo (FN)
            posM = posFN
            hayErrorDetectado = True
            if generaImagenComparacion:
              draw_box(comp_draw, resObj[2], 4, (255,0,0)) # cuadros Rojos

        else:

            # Objeto de misma Clase entre XML y Modelo con calc_IoU(r1, r2) < coefIoU -> Falso Positivo (FP)
            # Objeto de distinta Clase entre XML y Modelo con calc_IoU(r1, r2) ≥ coefIoU -> Falso Positivo (FP)
            # Objeto detectado por el Modelo que no aparece en el Modelo -> Falso Positivo (FP)
            posM = posFP
            hayErrorDetectado = True
            if generaImagenComparacion:            
              if resObj[1] == 999:
                draw_box(comp_draw, resObj[2], 4, (255,155,255)) # cuadros Rosa
              elif (resObj[0] != resObj[3]):
                draw_box(comp_draw, resObj[2], 4, (255,255,0)) # cuadros Amarillo
              else:
                draw_box(comp_draw, resObj[2], 4, (255,155,0)) # cuadros Naranja

        metricasImag[posM] = metricasImag[posM] + 1

    # Agrega a la métrica general y detalle por tipo de imagen    
    if not ( tipoCaso in metricasGral_porTipoCaso ):
      # inicaliza la matriz por tipo de caso si corresponde
      metricasGral_porTipoCaso[tipoCaso] = [ 0, 0, 0, 0] 
    for i in range(4):
      # actualiza matriz general
      metricasGral[i] = metricasGral[i] + metricasImag[i]
      # actualiza matriz general por tipo de caso
      metricasGral_porTipoCaso[tipoCaso][i] = metricasGral_porTipoCaso[tipoCaso][i] + metricasImag[i]

    if hayErrorDetectado: 
          listaXMLConProblemas.append( xml_file )
    
    if generaImagenComparacion and ( (muestraDetalleComparacionEnImagen == "Todas") or ( hayErrorDetectado and (muestraDetalleComparacionEnImagen == "Solo con Error") ) ):

        imMostrar = ImPIL.fromarray(np.array(comp_image_pil), 'RGB')
        display( imMostrar )
        print(" Nota colores:")
        print("     Verdaderos Positivos (igual ubicación y misma clase): cuadros en VERDE.")
        print("     Falsos Positivos (misma ubicación y distinta clase): cuadro en  AMARILLO")
        print("     Falsos Positivos (diferencia de ubicación y misma clase): cuadros en NARANJA")
        print("     Falsos Positivos (detectados de más por el Modelo): cuadros en ROSA ")        
        print("     Falsos Negativos (no detectado por el Modelo): cuadros en ROJO.")

    if ( (muestraDetalleMetricasPorImagen == "Todas") or ( hayErrorDetectado and (muestraDetalleMetricasPorImagen == "Solo con Error") ) ):
        mostrarMetricas(metricasImag, "Matriz de Confusión para la Imagen" )    
    else:
        if hayErrorDetectado: 
            print(" -- se detecta al menos un error!")
        else:
            print(" ++ sin error detectado.")

print("\n == VALIDACIÓN FINALIZADA ==")
print(" XMLs revisados: ", cantXMLProcesados)

auxLista = '['
if len(listaXMLConProblemas)>0:
  for f in listaXMLConProblemas:
      auxLista = auxLista + " '" + f + "',"
  auxLista = auxLista[:len(auxLista)-1] 
auxLista = auxLista + ']'    
print("\n ", len(listaXMLConProblemas)," XMLs con problemas detectados: \n\t", auxLista, "\n")


In [ ]:
#@title Mostrar las Métricas Generales
print("\n\n===========================================================================================================\n")
mostrarMetricas(metricasGral, "Matriz de Confusión General del Modelo Entrenado" )
# Muestra tiempo de ejecución promedio
if auxCantProc>0:
  print("\n\n# Ejecutar el modelo demora en promedio ",  (auxSumaTiempo/auxCantProc), " segundos. \n\n")
print("\n===========================================================================================================\n")

# muestra reporte de clasificación
if len(classObjReal)>0 and len(classObjModelo)>0:
  print("\n Reporte de Clasificación por Clase del Modelo Entrenado: ")
  print(classification_report(y_true=classObjReal, y_pred=classObjModelo))

print("\n===========================================================================================================\n")


In [ ]:
#@title Mostrar la Matriz de Confusión General por Clase (scrolleable)

###pd.set_option("display.max_rows", None, "display.max_columns", None)
# muestra matriz de confusion por clase
CLASSES = list(set(classObjReal + classObjModelo)) 
if len(CLASSES)==0:
  print("No se definieron los datos para generar matriz!")
  cmtx = ""
else:   
  print('\nMatriz de Confusión por Clase del Modelo Entrenado: ')
  cm = confusion_matrix(y_true=classObjReal, y_pred=classObjModelo, labels=CLASSES)
  cmtx = pd.DataFrame(
      cm, 
      index=['r:{:}'.format(x) for x in CLASSES], 
      columns=['m:{:}'.format(x) for x in CLASSES]
    )
  ###print(cmtx)
cmtx

In [ ]:
#@title Mostrar las Matrices de Confusión por Tipo de Caso

arTiposCasos = list(metricasGral_porTipoCaso.keys())
arTiposCasos.sort()
# recorre por tipo de caso evaluado
for tipoCaso in arTiposCasos:
  titulo = "Matriz de Confusión del Modelo Entrenado para imágenes "
  if tipoCaso[0:1] == "DA":
    titulo = titulo + "CON DA"
  else:
    titulo = titulo + "SIN DA"
  if tipoCaso[3] == "R":
    titulo = titulo + "y TAMAÑO REDUCIDO de TRANSICIONES"
  elif tipoCaso[3] == "S":
    titulo = titulo + "y SIN TRANSICIONES"
  else:
    titulo = titulo + "y TAMAÑO NORMAL de TRANSICIONES"
  titulo = titulo + " ["+tipoCaso+"]:"
  print("\n-----------------------------------------------------------------------------------------------------------\n")
  mostrarMetricas(metricasGral_porTipoCaso[tipoCaso], titulo)

print("\n-----------------------------------------------------------------------------------------------------------\n")
